In [175]:
import cvxpy as cp
import numpy as np
from typing import NamedTuple
from tqdm.notebook import tqdm

In [176]:
class AuctionResult(NamedTuple):
    alloc: np.ndarray
    welfare: np.ndarray
    payment: np.ndarray

In [177]:
# auction: bids consist of matrix n x m
# welfare maximizing allocation consists of matrix 

In [178]:
def max_welfare_alloc(bids_mat, k, exact_demand=False):
    # returns allocation, util of allocation

    num_agents = bids_mat.shape[0]
    num_items = bids_mat.shape[1]
    alloc_matrix = cp.Variable((num_agents, num_items), nonneg=True)
    item_supply_constraints = cp.sum(alloc_matrix, axis=0) <= 1
    if exact_demand:
        agent_demand_constraints = cp.sum(alloc_matrix, axis=1) == k
    else:
        agent_demand_constraints = cp.sum(alloc_matrix, axis=1) <= k
    welfare = cp.sum((cp.multiply(bids_mat, alloc_matrix)))
    problem = cp.Problem(cp.Maximize(welfare), [item_supply_constraints, agent_demand_constraints])
    problem.solve()
    return (alloc_matrix.value, problem.value)

In [179]:
bids_mat = np.random.rand(5,5)

In [180]:
bids_mat

array([[0.57150899, 0.69393195, 0.35016818, 0.38422451, 0.49580557],
       [0.85797057, 0.64772162, 0.60040824, 0.43477126, 0.0697347 ],
       [0.10095554, 0.86373388, 0.58203383, 0.07000974, 0.96860844],
       [0.07183608, 0.129089  , 0.52638918, 0.41289616, 0.59084748],
       [0.34350986, 0.69432531, 0.33531344, 0.29505438, 0.59608434]])

In [181]:
def vcg_auction(bids_mat, k, exact_demand=False):
    main_alloc, max_welfare_total_util = max_welfare_alloc(bids_mat, k, exact_demand=exact_demand)
    payments = np.zeros(bids_mat.shape[0])
    player_utils = (bids_mat * main_alloc).sum(axis=1)
    num_agents = bids_mat.shape[0]
    for i in range(num_agents):
        dropped_bid_mat = np.delete(bids_mat, (i), axis=0)
        dropped_player_utils = np.delete(player_utils, (i), axis=0) # player utils under full auction
        new_alloc, new_total_util = max_welfare_alloc(dropped_bid_mat, k, exact_demand=exact_demand)
        new_agent_utils = (new_alloc*dropped_bid_mat).sum(axis=1) # player utils without agent i's bid
        payments[i] = (new_agent_utils - dropped_player_utils).sum() 
    return AuctionResult(main_alloc, player_utils, payments)

In [182]:
vcg_auction(bids_mat, 2, exact_demand=False).payment.sum()

2.7725323260794825

In [185]:
rand_bid_mats = [np.random.rand(3,3) for _ in range(1000)]

In [186]:
payments = []
for mat in tqdm(rand_bid_mats):
    payments.append(vcg_auction(mat, 2, exact_demand=False).payment.sum())

In [191]:
np.mean(payments)

1.4227194834819876